In [1]:
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.express as px

In [2]:
import numpy as np
import pandas as pd

In [3]:
import pulp

In [4]:
f_conv_mon = 0.0008

In [5]:
def bmatrix(a):
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [6]:
C = np.array([[105, 1267, 1134, 2758, 515, 1350, 1311, 1750, 1565, 1137, 3110, 1365]])

C = C * f_conv_mon*1000

#C = np.array([[104.88, 140, 125.2, 90.96, 248.8, 109.2, 104.88, 140, 125.2, 90.96, 248.8, 109.2]])/100

In [7]:
C*1000

array([[  84000., 1013600.,  907200., 2206400.,  412000., 1080000.,
        1048800., 1400000., 1252000.,  909600., 2488000., 1092000.]])

In [8]:
from unicodedata import decimal
df = pd.DataFrame(C*1000)
df.to_clipboard(index=False,header=False, decimal=',')

In [9]:
A = np.array([  [     1,       1,       1,       1,       1,       1,       0,       0,       0,       0,       0,       0],
                [     0,       0,       0,       0,       0,       0,       1,       1,       1,       1,       1,       1],
                [   100,      50,      50,      50,     100,     100,      50,     100,     100,     100,     100,     100],
                [   100,      50,      50,      25,      50,     150,      50,     100,     100,      50,      50,      50],
                [     0,       0,       0,       0,       0,      12,       0,       0,      12,       0,       0,       0],
                [     1,       1,       2,       1,       1,       3,       1,       2,       2,       2,       3,       1],
                [     6,      29,      32,      14,      54,      60,       0,       0,      18,      13,      29,       8],
                [     2,       0,      17,      23,       0,       0,      22,       6,      50,     102,      77,      31],
                [     0,       0,       0,       0,       0,      12,      35,      24,      80,       0,     112,      58],
                [     9,       8,       7,      34,      26,      22,       0,       0,      52,       0,      43,       0],
                [   1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5],
                [     1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1],
                [   0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5],
                [   0.5,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0],
                [   390,     355,     370,     430,     440,       0,     275,       0,     100,     133,     213,     100],
                [   275,       0,     470,     305,       0,       0,    1430,     188,    1015,     955,    1343,    1215],
                [     0,       0,     150,       0,       0,     750,     443,     840,    1683,       0,    1748,    1095],
                [   300,     295,     305,     303,     210,     860,       0,       0,       0,       0,       0,       0],
                [345000,  233000,  216000,  242000,  360000,  650000,  244000,  242000,  310000,  263000,  250000,  285000]])

In [10]:
A[18,:] = A[18,:] * f_conv_mon

In [11]:
np.set_printoptions(precision = 3, suppress = True)
print(bmatrix(A[14:20, 0:12]))


\begin{bmatrix}
  390. & 355. & 370. & 430. & 440. & 0. & 275. & 0. & 100. & 133.\\
  213. & 100.\\
  275. & 0. & 470. & 305. & 0. & 0. & 1430. & 188. & 1015. & 955.\\
  1343. & 1215.\\
  0. & 0. & 150. & 0. & 0. & 750. & 443. & 840. & 1683. & 0.\\
  1748. & 1095.\\
  300. & 295. & 305. & 303. & 210. & 860. & 0. & 0. & 0. & 0.\\
  0. & 0.\\
  276. & 186.4 & 172.8 & 193.6 & 288. & 520. & 195.2 & 193.6 & 248. & 210.4\\
  200. & 228.\\
\end{bmatrix}


In [12]:
b = np.array([2395,2260,419190,397560,7000,13075,50593,554800,562400,562400,103056,103056,103056,954,20699716,20929709,21159705,21159705,1445855000])

#b = np.array([2395,2260,419190,397560,7000,13075,50593,554800,562400,562400,103056,103056,103056,954,20699716,20929709,21159705,21159705,1156684])

In [13]:
b[18] = b[18] * f_conv_mon

In [14]:
print(bmatrix(b))

\begin{bmatrix}
  2395 & 2260 & 419190 & 397560 & 7000 & 13075 & 50593 & 554800\\
  562400 & 562400 & 103056 & 103056 & 103056 & 954 & 20699716 & 20929709\\
  21159705 & 21159705 & 1156684\\
\end{bmatrix}


In [15]:
def create_var():
    var = np.array([])

    lbx = [0] * 12
    lbx[0] = 1500


    for i in range(1, 13):
        var = np.append(var, np.array([pulp.LpVariable("x{}".format(i), lowBound = lbx[i-1], upBound=None)]))
    
    return var

In [16]:
def objective():
    var = create_var()

    fo = C.dot(var)[0]

    return (var, fo)

In [17]:
def print_solution(sol, linearProblem, X):
    saida = "\nStatus = {} \nValue = {}".format(pulp.LpStatus[sol], pulp.value(linearProblem.objective))
    
    for i in range(0, X.shape[0]):
        saida += "\n{} = {}".format(X[i], pulp.value(X[i]))
    
    print(saida)

In [18]:
d_var, f = objective()
model = pulp.LpProblem("First_Model", pulp.LpMaximize)

model += f 

Ax = A.dot(d_var)

for i in range(0, Ax.shape[0]):
    model += Ax[i] <= b[i]

In [19]:
model.writeLP('models\original_problem_model_1y.lp')
solution = model.solve(pulp.GLPK(path='F:\glpk-4.65\w64\glpsol.exe', options=['--ranges models\sensit_orig_1y.sen']))

In [20]:
print_solution(sol=solution, linearProblem=model, X=d_var)


Status = Optimal 
Value = 6344131.5200000005
x1 = 1500.0
x2 = 0.0
x3 = 0.0
x4 = 895.0
x5 = 0.0
x6 = 0.0
x7 = 31.2
x8 = 1226.63
x9 = 0.0
x10 = 0.0
x11 = 1002.17
x12 = 0.0


In [21]:
for v in model.variables():
    print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)

x1 = 1500.0 	Reduced Cost = None
x10 = 0.0 	Reduced Cost = None
x11 = 1002.17 	Reduced Cost = None
x12 = 0.0 	Reduced Cost = None
x2 = 0.0 	Reduced Cost = None
x3 = 0.0 	Reduced Cost = None
x4 = 895.0 	Reduced Cost = None
x5 = 0.0 	Reduced Cost = None
x6 = 0.0 	Reduced Cost = None
x7 = 31.2 	Reduced Cost = None
x8 = 1226.63 	Reduced Cost = None
x9 = 0.0 	Reduced Cost = None


In [22]:
print(bmatrix(C*100))

\begin{bmatrix}
  8400. & 101360. & 90720. & 220640. & 41200. & 108000. & 104880. & 140000. & 125200.\\
  90960. & 248800. & 109200.\\
\end{bmatrix}


In [23]:
#from unicodedata import decimal
#df = pd.DataFrame(A)
#df.to_clipboard(index=False,header=False, decimal=',')